In [ ]:
import API_helper.url_module as hp
import pandas as pd
import time

import datetime
import os
from prefect.deployments import Deployment
from prefect import flow,task
from google.cloud import storage
from config import query_bq 
from prefect_gcp import GcpCredentials
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential.google_cred


@task(name='Get_BQ_SQL',log_prints=True)
def GetBQdata(query): 
    gcp_credentials_block = GcpCredentials.load("ete-projectcredential")
    df_bq = pd.read_gbq(query=query,
    project_id='ete-projectdatatalkclub',
    credentials=gcp_credentials_block.get_credentials_from_service_account()
    )
    return df_bq


@task(log_prints=True)
def getApiMap(para_regiontype,para_seriesgroup,para_season,para_unit,para_frequency,para_mindate,para_maxdate):

        print('Calling the API.')
        result = hp.map.regional(
            file_type ='json' \
            ,series_group = para_seriesgroup \
            ,region_type = para_regiontype \
            ,date = para_maxdate \
            ,start_date = para_mindate
            ,season=para_season \
            ,units=para_unit
            ,frequency = para_frequency
        )
        
        if result.ok:
            try:
                data_all = result.json()
                data_date =data_all['meta']['data']
                return data_date    

            except Exception:
                print(f'An error occured {Exception}')
                return None

        else:
            print(f'An error occur {result.status_code}.')
            print(f'An error occur {result.content}.')
            print('Waiting for 5 sec.')
            return 'retry'

@task(log_prints=True)
def movearchive(bucket,type):
    file_acrhive=  list(bucket.list_blobs())
    for file in file_acrhive:
        name = file.name
        pos = name.find('/')
        type = name[:pos]
        if type == 'staging':
            sub_pos = name.find('/',pos+1)
            sub_type = name[pos+1:sub_pos]
            if sub_type == type:
                bucket.copy_blob(file,destination_bucket=bucket,new_name=name.replace('staging','archive'))
                bucket.delete_blob(name)    

 



In [ ]:
@flow(log_prints=True)
def test():
    # series_parameter = getsqldata(query_sql.query_getMapPara)
    series_parameter = GetBQdata(query_bq.query_getMapPara)
    client = storage.Client()
    bucket = client.get_bucket('fred-etedata')
    print('Moving file to archive folder')
    movearchive(bucket,'map')
    time_stamp = datetime.datetime.now().strftime('%Y-%m-%d')
    for api_para in series_parameter.values.tolist():
        para_regiontype  = api_para[0][0]
        para_seriesgroup = api_para[1][0]
        para_season      = api_para[2][0]
        para_unit        = api_para[3][0]
        para_frequency   = api_para[4][0]   
        para_mindate     = api_para[5]
        para_maxdate     = api_para[6]

        retry = 0
        while retry <3:
            data_date =   getApiMap(para_regiontype,para_seriesgroup,para_season,para_unit, para_frequency,para_mindate,para_maxdate)
            if data_date == 'retry':
                retry+= 1
                print(f'Retry the current call. Current retry {retry}')
                continue
            elif data_date ==None:
                print(f'Skpping the call for {para_seriesgroup}')
                continue
            else:
                break
        
        if  retry ==3:
            print(f'Error occur skipping {para_seriesgroup}')
            continue
        time_para = para_maxdate.strftime('%Y-%m-%d')
        df_map = pd.json_normalize(data_date[time_para])
        
        
        uppload_path =f'staging/map/{time_stamp}/MapData_{para_seriesgroup}_{time_para}.parquet'
        print(f'Uploading file to cloud for {para_seriesgroup}')        
        bucket.blob(uppload_path).upload_from_string(df_map.to_parquet(), 'text/parquet')


        # retry = 0
        # while retry <2:
        #     try:
        #         bucket.blob(uppload_path).upload_from_string(df_map.to_parquet(), 'text/parquet')
        #         continue

        #     except Exception as err:
        #         retry+=1
        #         print(f'An error occur while upload file {err}')
        #         print(f'Wait for 5 second and retry. Current retry {retry}')
        #         time.sleep(5)
        time.sleep(1)
    print('Finish running the function.')
# test()

In [ ]:
@flow
def crdtest():
    from dotenv import load_dotenv
    from prefect_gcp import GcpCredentials

    basedir=os.getcwd()
    load_dotenv(os.path.join(basedir, './.env'))

    gcp_credentials_block = GcpCredentials.load(os.getenv("Prefect_Credential")).get_credentials_from_service_account()
    return gcp_credentials_block
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = gcp_credentials_block.get_credentials_from_service_account()

In [ ]:
cred =crdtest()

In [ ]:
cred.__dict__

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = cred.__dict__.items()


In [ ]:
cred

In [ ]:

file_acrhive=  list(bucket.list_blobs())
for file in file_acrhive:
    name = file.name
    pos = name.find('/')
    type = name[:pos]
    if type == 'staging':
        sub_pos = name.find('/',pos+1)
        sub_type = name[pos+1:sub_pos]
        if sub_type == type:
            bucket.copy_blob(file,destination_bucket=bucket,new_name=name.replace('staging','archive'))
            bucket.delete_blob(name)   

In [ ]:
from dotenv import load_dotenv


basedir=os.getcwd()[:-5]
load_dotenv(os.path.join(basedir, './.env'))

In [ ]:
from prefect_gcp.cloud_storage import GcsBucket

@flow(name='my_unique_name')
def test():
    gcp_cloud_storage_bucket_block = GcsBucket.load("ete-project-gcs")
    gcp_cloud_storage_bucket_block.UPLOAD_FROM_DATAFRAME()

In [ ]:
from prefect_gcp.credentials import GcpCredentials
from prefect import flow,task
@flow()
def test():
    gcp_credentials_block = GcpCredentials.load("ete-projectcredential")
    gcp_credentials_block.get_credentials_from_service_account()
    print(gcp_credentials_block)

In [ ]:
test()

In [ ]:
def create_bigquery_client():
    gcp_credentials_block = GcpCredentials.load("BLOCK_NAME")
    google_auth_credentials = gcp_credentials_block.get_credentials_from_service_account()
    bigquery_client = bigquery.Client(credentials=google_auth_credentials)

In [ ]:
GcpCredentials.get_credentials_from_service_account()

In [ ]:
gcp_credentials_block.client_type()

In [ ]:
gcp_credentials_block.service_account_file()

In [ ]:
from prefect_gcp.secret_manager import read_secret

In [ ]:
from prefect_gcp import GcpCredentials
gcp_credentials_block = GcpCredentials.load("ete-projectcredential")

In [ ]:
gcp_credentials_block

In [ ]:
from prefect import flow
from prefect_gcp import GcpCredentials
from prefect_gcp.secret_manager import read_secret

@flow()
def example_cloud_storage_read_secret_flow():
    gcp_credentials = GcpCredentials.load("ete-projectcredential")
    secret_value = read_secret("secret_name", gcp_credentials, version_id=1)
    return secret_value

example_cloud_storage_read_secret_flow()


In [ ]:
import API_helper.url_module as hp
import pandas as pd
import datetime
import time
import os
from prefect import flow,task
from google.cloud import storage
from config import query_bq,clean_df
from prefect.deployments import Deployment
from prefect_gcp import GcpCredentials
from dotenv import load_dotenv

@task(log_prints=True)
def movearchive(bucket,typecheck):
    file_acrhive=  list(bucket.list_blobs())
    return file_acrhive
    for file in file_acrhive:
        name = file.name
        pos = name.find('/')
        type = name[:pos]
        if type == 'staging':
            sub_pos = name.find('/',pos+1)
            sub_type = name[pos+1:sub_pos]
            if sub_type == type:
                bucket.copy_blob(file,destination_bucket=bucket,new_name=name.replace('staging','archive'))
                bucket.delete_blob(name)    
@flow(log_prints=True)
def main():
    basedir=os.getcwd()
    load_dotenv(os.path.join(basedir, './.env'))
    gcp_credentials_block = GcpCredentials.load(os.getenv("Prefect_Credential"))

    # Get series id base on cat id
    client = storage.Client(credentials=gcp_credentials_block.get_credentials_from_service_account())
    bucket = client.get_bucket(os.getenv("Gcs_Bucket_name"))
    
    a =movearchive(bucket,'series')
    return a

In [ ]:
os.getenv("Google_Cred_path")

In [ ]:
basedir

In [ ]:
def GetBQdata(query):     
    df_bq = pd.read_gbq(query=query,
    project_id=os.getenv("Gcp_Project_id"),
    credentials=gcp_credentials_block.get_credentials_from_service_account()
    )

In [ ]:
client = storage.Client()

In [ ]:
typecheck ='series'

In [ ]:
typecheck

In [ ]:
for file in list:
        name = file.name
        pos = name.find('/')
        type = name[:pos]
        if type == 'staging':
            sub_pos = name.find('/',pos+1)
            sub_type = name[pos+1:sub_pos]
            if sub_type == typecheck:
                print(file)
                # bucket.copy_blob(file,destination_bucket=bucket,new_name=name.replace('staging','archive'))
                # bucket.delete_blob(name)    

In [ ]:
trigger_dbt_debug('dev')